# Predict on new data using a trained CNN on XPS data on Google Colab

In this notebook, we will use a trained convolutional network to predict on unseen XPS spectra.

## Setup

### Mount google drive, change working directory

In [ ]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Change working path
os.chdir('/content/drive/My Drive/app')

### Install packages and import modules

In [ ]:
%%capture
# Install packages
!pip install python-docx
!pip install tensorflow==2.3.0 as tf # to be dropped when tf 2.4 works

# Import standard modules and magic commands
import datetime
import numpy as np
import pytz
import importlib
import matplotlib.pyplot as plt

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

### Check TensorFlow version

In [ ]:
tf.__version__

## Predict on new data set

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('\n Modules were reloaded.')
except:
    import xpsdeeplearning.network.classifier as classifier
    import xpsdeeplearning.network.utils as clfutils
    print('\n Modules were loaded.')

### Set up the parameters & folder structure

In [ ]:
np.random.seed(502)
time = datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
exp_name = 'Pd_2_classes_measured_predict_using_20210120_20h36m_after_528_epochs'

clf = classifier.Classifier(time = time,
                            exp_name = exp_name,
                            task = 'regression',
                            intensity_only = True)

### Load and inspect the data

In [ ]:
input_filepath = r'/content/drive/My Drive/app/datasets/20210125_palladium_measured_caro_fit.h5'

train_test_split = 0.99
train_val_split = 0
no_of_examples = 180

        
X_train, X_val, X_test, y_train, y_val, y_test,\
    names_train, names_val, names_test =\
        clf.load_data_preprocess(input_filepath = input_filepath,
                                 no_of_examples = no_of_examples,
                                 train_test_split = train_test_split,
                                 train_val_split = train_val_split)
        
# Check how the examples are distributed across the classes.
class_distribution = clf.check_class_distribution()
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 10, dataset = 'test')  

### Load the model

In [ ]:
clf.load_model(model_path = '/content/drive/My Drive/app/saved_models/20210120_20h36m_Pd_2_classes_linear_comb_new_noise')

### Compile and summarize the model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError, MeanSquaredError, CategoricalCrossentropy

learning_rate = 1e-05
optimizer = Adam(learning_rate = learning_rate) 

if clf.task == 'regression':
    mae = MeanAbsoluteError()
    clf.model.compile(loss = mae, optimizer = optimizer)
    # =============================================================================
    # mse = MeanSquaredError()
    # clf.model.compile(loss = mse, optimizer = optimizer)
    # =============================================================================
    
if clf.task == 'classification':
    categorical_crossentropy = CategoricalCrossentropy()
    clf.model.compile(loss = categorical_crossentropy,
                      optimizer = optimizer,
                      metrics = ['accuracy'])

# Plot summary and save model plot.
clf.summary()
clf.save_and_print_model_image()

### Evaluate on test data

In [ ]:
clf.batch_size = 32

if clf.task == 'classification':
    score = clf.evaluate()
    test_loss, test_accuracy = score[0], score[1]
    print('Test loss: ' + str(np.round(test_loss, decimals=8)))
    print('Test accuracy: ' + str(np.round(test_accuracy, decimals=3)))
elif clf.task == 'regression':
    test_loss = clf.evaluate()
    print('Test loss: ' + str(np.round(test_loss, decimals=8)))

###  Predict on train & test data

In [ ]:
pred_train, pred_test = clf.predict()
if clf.task == 'classification':
    pred_train_classes, pred_test_classes = clf.predict_classes()

### Show some predictions on random test samples

In [ ]:
clf.plot_random(no_of_spectra = 15, dataset = 'test', with_prediction = True)  


### Show the worst predictions on the test samples

In [ ]:
clf.show_worst_predictions(no_of_spectra = 20)

### Save data

In [ ]:
clf.save_hyperparams()
clf.shelve_results(full = False)

## Check where and why the predictions fail

### Calculate loss for each example in the test set

In [ ]:
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()
mape = tf.keras.losses.MeanAbsolutePercentageError()

mse_losses = [mse(clf.datahandler.y_test[i], clf.datahandler.pred_test[i]).numpy() \
                  for i in range(clf.y_test.shape[0])]
mae_losses = [mae(clf.datahandler.y_test[i], clf.datahandler.pred_test[i]).numpy() \
                  for i in range(clf.y_test.shape[0])]
mape_losses = [mape(clf.datahandler.y_test[i], clf.datahandler.pred_test[i]).numpy() \
                  for i in range(clf.y_test.shape[0])]

In [ ]:
def show_worst_of_one_kind(indices, spectra = [0,20]):
    if len(indices) > 0:
        data = []
        texts = []
        
        X = clf.datahandler.X_test
        y = clf.datahandler.y_test
        
        no_of_spectra =  spectra[1] - spectra[0]
                     
        for i in range(no_of_spectra):
            index = indices[spectra[0]+i]
            if self.intensity_only:
                new_energies = np.reshape(np.array(clf.datahandler.energies),(-1,1))
                data.append(np.hstack((new_energies, X[index])))
            else:
                data.append(X[index])
            
            label = str(np.around(clf.datahandler.y_test[index], decimals = 3))
            full_text = 'Real: ' +  label + '\n'
             
            tmp_array = np.around(clf.datahandler.pred_test[index], decimals = 3) 
            full_text += 'Prediction: ' + str(list(tmp_array)) + '\n'
            try:
                full_text += clf.datahandler._write_aug_text(dataset = 'test',
                                                 index = index)
            except AttributeError:
                pass
            try:
                full_text += clf.datahandler._write_measured_text(dataset = 'test',
                                                      index = index)
            except AttributeError:
                 pass
            loss_text = ('MSE: ' + str(np.around(mse_losses[index], decimals = 3)) + ', ' +\
                         'MAE: ' + str(np.around(mae_losses[index], decimals = 3)))
            
            text = full_text + loss_text
            texts.append(text)
            
        data = np.array(data)
        
        graphic = SpectraPlot(data=data,
                              annots=texts)
        fig, axs = graphic.plot()

### Show worst predictions for single spectra

In [ ]:
worst_indices_single = [j[1] for j in sorted([(x,i) for (i,x) in \
                                        enumerate(mae_losses) if len(np.where(clf.y_test[i] == 0.)[0]) == 3],
                                        reverse=True)] 
show_worst_of_one_kind(worst_indices_single, spectra = [0,20])

### Show worst predictions for different loss thresholds (single spectra)


In [ ]:
threshold = 0.2
worst_indices_single_01 = [j[1] for j in sorted([(x,i) for (i,x) in \
                           enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) == 3 and x >= threshold)],
                           reverse=True)]
  
show_worst_of_one_kind(worst_indices_single_01,
                       spectra = [len(worst_indices_single_01)-10,
                                  len(worst_indices_single_01)])

print('{0} of {1} test samples with single species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_single_01)),
    str(len(worst_indices_single)),
    str(100*(np.around(len(worst_indices_single_01)/len(worst_indices_single), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.1
worst_indices_single_02 = [j[1] for j in sorted([(x,i) for (i,x) in \
                        enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) == 3 and x >= threshold)],
                        reverse=True)]

show_worst_of_one_kind(worst_indices_single_02,
                       spectra = [len(worst_indices_single_02)-10,
                                  len(worst_indices_single_02)])

print('{0} of {1} test samples with single species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_single_02)),
    str(len(worst_indices_single)),
    str(100*(np.around(len(worst_indices_single_02)/len(worst_indices_single), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.05
worst_indices_single_03 = [j[1] for j in sorted([(x,i) for (i,x) in \
                        enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) == 3 and x >= threshold)],
                        reverse=True)]

show_worst_of_one_kind(worst_indices_single_03,
                       spectra = [len(worst_indices_single_03)-10,
                                  len(worst_indices_single_03)])

print('{0} of {1} test samples with single species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_single_03)),
    str(len(worst_indices_single)),
    str(100*(np.around(len(worst_indices_single_03)/len(worst_indices_single), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.02
worst_indices_single_04 = [j[1] for j in sorted([(x,i) for (i,x) in \
                        enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) == 3 and x >= threshold)],
                        reverse=True)]

show_worst_of_one_kind(worst_indices_single_04,
                       spectra = [len(worst_indices_single_04)-10,
                                  len(worst_indices_single_04)])

print('{0} of {1} test samples with single species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_single_04)),
    str(len(worst_indices_single)),
    str(100*(np.around(len(worst_indices_single_04)/len(worst_indices_single), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.01
worst_indices_single_05 = [j[1] for j in sorted([(x,i) for (i,x) in \
                        enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) == 3 and x >= threshold)],
                        reverse=True)]

show_worst_of_one_kind(worst_indices_single_05,
                       spectra = [len(worst_indices_single_05)-10,
                                  len(worst_indices_single_05)])

print('{0} of {1} test samples with single species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_single_05)),
    str(len(worst_indices_single)),
    str(100*(np.around(len(worst_indices_single_05)/len(worst_indices_single), decimals = 3))),
    str(threshold)))

### Show worst predictions for linearly combined spectra

In [ ]:
worst_indices_multiple = [j[1] for j in sorted([(x,i) for (i,x) in \
                          enumerate(mae_losses) if len(np.where(clf.y_test[i] == 0.)[0]) != 3],
                          reverse=True)] 
show_worst_of_one_kind(worst_indices_multiple, spectra = [0,20])

### Show worst predictions for different loss thresholds (linearly combined spectra)

In [ ]:
threshold = 0.3
worst_indices_multiple_01 = [j[1] for j in sorted([(x,i) for (i,x) in \
                             enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) != 3 and x >= threshold)],
                             reverse=True)]  
                                             
show_worst_of_one_kind(worst_indices_multiple_01,
                       spectra = [len(worst_indices_multiple_01)-10,
                                 len(worst_indices_multiple_01)])

print('{0} of {1} test samples with multiple species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_multiple_01)),
    str(len(worst_indices_multiple)),
    str(100*(np.around(len(worst_indices_multiple_01)/len(worst_indices_multiple), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.2
worst_indices_multiple_02 = [j[1] for j in sorted([(x,i) for (i,x) in \
                            enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) != 3 and x >= threshold)],
                            reverse=True)]     

show_worst_of_one_kind(worst_indices_multiple_02,
                           spectra = [len(worst_indices_multiple_02)-10,
                                      len(worst_indices_multiple_02)])


print('{0} of {1} test samples with multiple species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_multiple_02)),
    str(len(worst_indices_multiple)),
    str(100*(np.around(len(worst_indices_multiple_02)/len(worst_indices_multiple), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.1
worst_indices_multiple_03 = [j[1] for j in sorted([(x,i) for (i,x) in \
                             enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) != 3 and x >= threshold)],
                             reverse=True)]                  

if (len(worst_indices_multiple_03)) > 10:
    show_worst_of_one_kind(worst_indices_multiple_03,
                           spectra = [len(worst_indices_multiple_03)-10,
                                      len(worst_indices_multiple_03)])
else:
    show_worst_of_one_kind(worst_indices_multiple_03,
                           spectra = [0,
                                      len(worst_indices_multiple_03)])

print('{0} of {1} test samples with multiple species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_multiple_03)),
    str(len(worst_indices_multiple)),
    str(100*(np.around(len(worst_indices_multiple_03)/len(worst_indices_multiple), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.05
worst_indices_multiple_04 = [j[1] for j in sorted([(x,i) for (i,x) in \
                             enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) != 3 and x >= threshold)],
                             reverse=True)]                  

if (len(worst_indices_multiple_04)) > 10:
    show_worst_of_one_kind(worst_indices_multiple_04,
                           spectra = [len(worst_indices_multiple_04)-10,
                                      len(worst_indices_multiple_04)])
else:
    show_worst_of_one_kind(worst_indices_multiple_04,
                           spectra = [0,
                                      len(worst_indices_multiple_04)])

print('{0} of {1} test samples with multiple species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_multiple_04)),
    str(len(worst_indices_multiple)),
    str(100*(np.around(len(worst_indices_multiple_04)/len(worst_indices_multiple), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.025
worst_indices_multiple_05 = [j[1] for j in sorted([(x,i) for (i,x) in \
                             enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) != 3 and x >= threshold)],
                             reverse=True)]                  

if (len(worst_indices_multiple_05)) > 10:
    show_worst_of_one_kind(worst_indices_multiple_05,
                           spectra = [len(worst_indices_multiple_05)-10,
                                      len(worst_indices_multiple_05)])
else:
    show_worst_of_one_kind(worst_indices_multiple_05,
                           spectra = [0,
                                      len(worst_indices_multiple_05)])

print('{0} of {1} test samples with multiple species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_multiple_05)),
    str(len(worst_indices_multiple)),
    str(100*(np.around(len(worst_indices_multiple_05)/len(worst_indices_multiple), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.01
worst_indices_multiple_06 = [j[1] for j in sorted([(x,i) for (i,x) in \
                             enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) != 3 and x >= threshold)],
                             reverse=True)]                  

if (len(worst_indices_multiple_06)) > 10:
    show_worst_of_one_kind(worst_indices_multiple_06,
                           spectra = [len(worst_indices_multiple_06)-10,
                                      len(worst_indices_multiple_06)])
else:
    show_worst_of_one_kind(worst_indices_multiple_06,
                           spectra = [0,
                                      len(worst_indices_multiple_06)])

print('{0} of {1} test samples with multiple species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_multiple_06)),
    str(len(worst_indices_multiple)),
    str(100*(np.around(len(worst_indices_multiple_06)/len(worst_indices_multiple), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.005
worst_indices_multiple_07 = [j[1] for j in sorted([(x,i) for (i,x) in \
                             enumerate(mae_losses) if (len(np.where(clf.y_test[i] == 0.)[0]) != 3 and x >= threshold)],
                             reverse=True)]                  

if (len(worst_indices_multiple_07)) > 10:
    show_worst_of_one_kind(worst_indices_multiple_07,
                           spectra = [len(worst_indices_multiple_07)-10,
                                      len(worst_indices_multiple_07)])
else:
    show_worst_of_one_kind(worst_indices_multiple_07,
                           spectra = [0,
                                      len(worst_indices_multiple_07)])

print('{0} of {1} test samples with multiple species ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_multiple_07)),
    str(len(worst_indices_multiple)),
    str(100*(np.around(len(worst_indices_multiple_07)/len(worst_indices_multiple), decimals = 3))),
    str(threshold)))

### Show worst predictions for all

In [ ]:
worst_indices = [j[1] for j in sorted([(x,i) for (i,x) in enumerate(mae_losses)],
                          reverse=True)] 
show_worst_of_one_kind(worst_indices, spectra = [0,20])

### Show worst predictions for different loss thresholds (all spectra)

In [ ]:
threshold = 0.3
worst_indices_01 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]  
                                             
show_worst_of_one_kind(worst_indices_01,
                       spectra = [len(worst_indices_01)-10,
                                  len(worst_indices_01)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_01)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_01)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.2
worst_indices_02 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]  
                                             
show_worst_of_one_kind(worst_indices_02,
                       spectra = [len(worst_indices_02)-10,
                                  len(worst_indices_02)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_02)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_02)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.1
worst_indices_03 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_03)) > 10:
    show_worst_of_one_kind(worst_indices_03,
                           spectra = [len(worst_indices_03)-10,
                                      len(worst_indices_03)])
else:
    show_worst_of_one_kind(worst_indices_03,
                           spectra = [0,
                                      len(worst_indices_03)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_03)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_03)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.05
worst_indices_04 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_04)) > 10:
    show_worst_of_one_kind(worst_indices_04,
                           spectra = [len(worst_indices_04)-10,
                                      len(worst_indices_04)])
else:
    show_worst_of_one_kind(worst_indices_04,
                           spectra = [0,
                                      len(worst_indices_04)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_04)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_04)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.025
worst_indices_05 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_05)) > 10:
    show_worst_of_one_kind(worst_indices_05,
                           spectra = [len(worst_indices_05)-10,
                                      len(worst_indices_05)])
else:
    show_worst_of_one_kind(worst_indices_05,
                           spectra = [0,
                                      len(worst_indices_05)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_05)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_05)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.01
worst_indices_06 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_06)) > 10:
    show_worst_of_one_kind(worst_indices_06,
                           spectra = [len(worst_indices_06)-10,
                                      len(worst_indices_06)])
else:
    show_worst_of_one_kind(worst_indices_06,
                           spectra = [0,
                                      len(worst_indices_06)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_06)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_06)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.005
worst_indices_07 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_07)) > 10:
    show_worst_of_one_kind(worst_indices_07,
                           spectra = [len(worst_indices_07)-10,
                                      len(worst_indices_07)])
else:
    show_worst_of_one_kind(worst_indices_07,
                           spectra = [0,
                                      len(worst_indices_07)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_07)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_07)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.001
worst_indices_08 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_08)) > 10:
    show_worst_of_one_kind(worst_indices_08,
                           spectra = [len(worst_indices_08)-10,
                                      len(worst_indices_08)])
else:
    show_worst_of_one_kind(worst_indices_08,
                           spectra = [0,
                                      len(worst_indices_08)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_08)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_08)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.0005
worst_indices_09 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_09)) > 10:
    show_worst_of_one_kind(worst_indices_09,
                           spectra = [len(worst_indices_09)-10,
                                      len(worst_indices_09)])
else:
    show_worst_of_one_kind(worst_indices_09,
                           spectra = [0,
                                      len(worst_indices_09)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_09)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_09)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.00025
worst_indices_10 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_10)) > 10:
    show_worst_of_one_kind(worst_indices_10,
                           spectra = [len(worst_indices_10)-10,
                                      len(worst_indices_10)])
else:
    show_worst_of_one_kind(worst_indices_10,
                           spectra = [0,
                                      len(worst_indices_10)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_10)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_10)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.0001
worst_indices_11 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]                 

if (len(worst_indices_11)) > 10:
    show_worst_of_one_kind(worst_indices_11,
                           spectra = [len(worst_indices_11)-10,
                                      len(worst_indices_11)])
else:
    show_worst_of_one_kind(worst_indices_11,
                           spectra = [0,
                                      len(worst_indices_11)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_11)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_11)/len(worst_indices), decimals = 3))),
    str(threshold)))

In [ ]:
threshold = 0.00001
worst_indices_12 = [j[1] for j in sorted([(x,i) for (i,x) in \
                                          enumerate(mae_losses) if x >= threshold],
                             reverse=True)]

if (len(worst_indices_12)) > 10:
    show_worst_of_one_kind(worst_indices_12,
                           spectra = [len(worst_indices_12)-10,
                                      len(worst_indices_12)])
else:
    show_worst_of_one_kind(worst_indices_12,
                           spectra = [0,
                                      len(worst_indices_12)])

print('{0} of {1} test samples ({2}%) have a mean absolute error of of at least {3}.'.format(
    str(len(worst_indices_12)),
    str(len(worst_indices)),
    str(100*(np.around(len(worst_indices_12)/len(worst_indices), decimals = 3))),
    str(threshold)))

## Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"),
            include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time
import os

time.sleep(20)
save_notebook()
print('Notebook saved!')
time.sleep(30)
current_file = '/content/drive/My Drive/app/xpsdeeplearning/predict_unknown_multiple.ipynb'
output_file = os.path.join(clf.log_dir,'predict_unknown_multiple_out.html')
output_HTML(current_file, output_file)
print('HTML file saved!')